In [1]:
%pylab
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid_at_node
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np
from matplotlib import colors,  cm
import matplotlib.animation as animation
from scipy.optimize import fsolve

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [12]:
mg = RasterModelGrid((5,5),100)
junc_elev = mg.add_zeros('node', 'junction__elevation')
R = mg.add_zeros('node', 'input__discharge')

d_h = mg.add_zeros('link','hydraulic__diameter')
mg.at_link['hydraulic__diameter'][mg.active_links]= 0.5*np.random.rand(mg.number_of_active_links)

h = mg.add_zeros('node', 'hydraulic__head')
thickness = 500.*np.ones(mg.number_of_nodes)
Z = mg.add_field('node', 'ice__thickness', thickness)
#a = mg.add_ones('link', 'head_loss__exponent')
#f = mg.add_ones('link', 'friction__factor')
#h = np.random.rand(mg.number_of_nodes)
Q = mg.add_zeros('link', 'conduit__discharge')
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)
#set boundary node head

#add input to node 310
moulin_Q=1.
mg.at_node['input__discharge'][7]=moulin_Q
#set heads at edges
h[mg.nodes_at_left_edge] = 400.
h[mg.nodes_at_right_edge] = 300.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
mg.set_closed_boundaries_at_grid_edges(False,True,True,True)
Q[mg.active_links] = 1.*0.1
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print "Number of links = ", mg.number_of_links
print "Number of nodes = ", mg.number_of_nodes
print "Number of active links = ", mg.number_of_active_links
print "Number of core nodes = ", mg.number_of_core_nodes

pfn = PresFlowNetwork(mg)
mc = MeltCreep(mg, dt=500.)

#code added for fsolve algorithm
#dhdx = mg.add_zeros('link', 'hydraulic_head_gradient', noclobber=False)
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)


Number of links =  40
Number of nodes =  25
Number of active links =  15
Number of core nodes =  9


In [3]:
def network_residuals(heads):
    h[mg.core_nodes] = heads
    mg.calc_grad_at_link(h, out=dhdx)
    Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    return mg.calc_net_flux_at_node(Q, out=net_node_flux)[mg.core_nodes]/mg.dx - mg.at_node['input__discharge'][mg.core_nodes]


In [4]:
mg.calc_grad_at_link(h, out=dhdx)


array([ 0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  3., -4.,  0.,  0.,  3.,
        0.,  0.,  0.,  0.,  0., -4.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,
        0., -4.,  0.,  0.,  3., -4.,  0.,  0.,  0., -3.,  0.,  0.,  0.,  0.])

In [12]:
mg.at_node['hydraulic__head'][mg.core_nodes] = fsolve(network_residuals, h[mg.core_nodes])


In [16]:
#mg.at_node['hydraulic__head']
figure()
imshow_grid_at_node(mg,h)

In [3]:
pfn.run_one_step_fsolve()



/home/mcoving/anaconda2/lib/python2.7/site-packages/landlab/components/conduit_networks/pressurized_flow.py:145: RuntimeWarning: divide by zero encountered in divide
  self.r = 8*f*L/(g*r_s**2.*d_h**5.)


In [10]:
mg.calc_net_flux_at_node(Q)[mg.core_nodes]/mg.dx - mg.at_node['input__discharge'][mg.core_nodes]

array([ 0.2, -0.9,  0.1,  0.1,  0. ,  0. ,  0. , -0.1, -0.1])

In [11]:
pfn.network_residuals(pfn.h[mg.core_nodes])

array([ -6.41283491e+00,  -3.90730957e+00,  -2.91971441e-03,
        -4.53709732e-01,  -2.33235889e-06,  -2.36074504e-07,
        -2.91908581e+00,  -2.43000897e+00,  -5.86055700e-04])

In [9]:
pfn.run_one_step()

Number of iterations = 1 tolerance = 4.14923967584
Number of iterations = 2 tolerance = 0.00092358931835


In [24]:
pfn.h

array([     0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,    400.        ,  11319.21951398,  11319.43804962,
        10330.85721235,    300.        ,    400.        ,  10205.65626153,
        11319.24415997,  10304.6107867 ,    300.        ,    400.        ,
         9900.5006292 ,    302.26946321,    301.90091924,    300.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ])

In [25]:
mg.at_node['hydraulic__head']

array([     0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,    400.        ,  11319.21951398,  11319.43804962,
        10330.85721235,    300.        ,    400.        ,  10205.65626153,
        11319.24415997,  10304.6107867 ,    300.        ,    400.        ,
         9900.5006292 ,    302.26946321,    301.90091924,    300.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ])

In [13]:
def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='jet', magnitude= False, lw=2):
    link_head_x = grid.node_x[grid.node_at_link_head]
    link_head_y = grid.node_y[grid.node_at_link_head]
    link_tail_x = grid.node_x[grid.node_at_link_tail]
    link_tail_y = grid.node_y[grid.node_at_link_tail]
    if magnitude:
        values = abs(grid.at_link[value_name])
    else:
        values = grid.at_link[value_name]
    
    #Normalize color values
    if autoscale:
        cnorm = colors.Normalize()
        cnorm.autoscale(values)
    else:
        cnorm = colors.Normalize(vmin,vmax)
    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
    scalarMap.set_array(values)
    #set_cmap()
    for i, value in enumerate(values):
        xs = [link_head_x[i],link_tail_x[i]]
        ys = [link_head_y[i],link_tail_y[i]]
        img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))
    colorbar(scalarMap)


In [14]:
nsteps = 500
step_start = 1
fig1, axs = subplots(2,2,figsize=(9,6))
every = 50
for step in arange(nsteps)+step_start:
    print "step =",step, " avg d_h=",mg.at_link['hydraulic__diameter'].mean()
    pfn.run_one_step_fsolve()    
    #mg.calc_grad_at_link(h, out=dhdx)
    #h = fsolve(network_residuals, h[mg.core_nodes])
    #Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    mc.run_one_step()
    if (step % every)==0: #make an animation frame
        subplot(2,2,1)
        imshow_grid_at_node(mg, h)
        #plot(mg.at_node['hydraulic__head'][mg.core_nodes])
        #ylim([300,400])
        subplot(2,2,2)
        plot_links(mg, 'conduit__discharge', magnitude=True)#, vmin=0., vmax=1.m autoscale=False)
        #plot(mg.at_link['conduit__discharge'][mg.active_links])
        #ylabel('')
        #ylim([0,1.5])
        subplot(2,2,3)
        plot_links(mg, 'hydraulic__diameter')#, autoscale=False, vmin=0, vmax=1.)
        subplot(2,2,4)
        plot(mg.at_link['hydraulic__diameter'][mg.active_links])
        #ylim([0.5,1.5])
        image_name = 'heads_and_discharge'+str(step).zfill(6)+'.png'
        tight_layout()
        savefig(image_name)
        fig1.clf()


step = 1  avg d_h= 0.158337480299
mean ddh =  6998.31431216
step = 2  avg d_h= 2624.52620454
mean ddh =  -23.2056704492
step = 3  avg d_h= 2615.82407812
mean ddh =  -23.1288933194
step = 4  avg d_h= 2607.15074313
mean ddh =  -23.0522999207
step = 5  avg d_h= 2598.50613066
mean ddh =  -22.9759288987
step = 6  avg d_h= 2589.89015732
mean ddh =  -22.8997933427
step = 7  avg d_h= 2581.30273482
mean ddh =  -22.8238989292
step = 8  avg d_h= 2572.74377272
mean ddh =  -22.7482483864
step = 9  avg d_h= 2564.21317957
mean ddh =  -22.6728430379
step = 10  avg d_h= 2555.71086344
mean ddh =  -22.5976834543
step = 11  avg d_h= 2547.23673214
mean ddh =  -22.5227697674
step = 12  avg d_h= 2538.79069348
mean ddh =  -22.4481018363
step = 13  avg d_h= 2530.37265529
mean ddh =  -22.3736793424
step = 14  avg d_h= 2521.98252554
mean ddh =  -22.2995018467
step = 15  avg d_h= 2513.62021234
mean ddh =  -22.2255688263
step = 16  avg d_h= 2505.28562403
mean ddh =  -22.1518796983
step = 17  avg d_h= 2496.97866915

/home/mcoving/anaconda2/lib/python2.7/site-packages/landlab/components/conduit_networks/glacier_conduit.py:133: RuntimeWarning: invalid value encountered in divide
  self.melt = 16.*self.rho_w*f/(np.pi**3.*self.rho_ice*self.L_v)*abs(self._grid.at_link['conduit__discharge'])**3./d_h**6.
/home/mcoving/anaconda2/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


mean ddh =  -20.5232710226
step = 40  avg d_h= 2313.33692446
mean ddh =  -20.4552135992
step = 41  avg d_h= 2305.66621936
mean ddh =  -20.3873817602
step = 42  avg d_h= 2298.0209512
mean ddh =  -20.3197747636
step = 43  avg d_h= 2290.40103566
mean ddh =  -20.2523918689
step = 44  avg d_h= 2282.80638871
mean ddh =  -20.185232338
step = 45  avg d_h= 2275.23692658
mean ddh =  -20.1182954347
step = 46  avg d_h= 2267.6925658
mean ddh =  -20.0515804249
step = 47  avg d_h= 2260.17322314
mean ddh =  -19.9850865768
step = 48  avg d_h= 2252.67881567
mean ddh =  -19.9188131606
step = 49  avg d_h= 2245.20926074
mean ddh =  -19.8527594485
step = 50  avg d_h= 2237.76447594
mean ddh =  -19.7869247153
step = 51  avg d_h= 2230.34437917
mean ddh =  -19.7213082377
step = 52  avg d_h= 2222.94888859
mean ddh =  -19.6559092947
step = 53  avg d_h= 2215.5779226
mean ddh =  -19.5907271674
step = 54  avg d_h= 2208.23139991
mean ddh =  -19.5257611393
step = 55  avg d_h= 2200.90923948
mean ddh =  -19.4610104961
s

step = 185  avg d_h= 1429.18714529
mean ddh =  -12.6365079359
step = 186  avg d_h= 1424.44845482
mean ddh =  -12.5946033054
step = 187  avg d_h= 1419.72547858
mean ddh =  -12.5528376574
step = 188  avg d_h= 1415.01816446
mean ddh =  -12.5112105309
step = 189  avg d_h= 1410.32646051
mean ddh =  -12.4697214666
step = 190  avg d_h= 1405.65031496
mean ddh =  -12.4283700067
step = 191  avg d_h= 1400.98967621
mean ddh =  -12.3871556948
step = 192  avg d_h= 1396.34449282
mean ddh =  -12.3460780762
step = 193  avg d_h= 1391.71471354
mean ddh =  -12.3051366974
step = 194  avg d_h= 1387.10028728
mean ddh =  -12.2643311067
step = 195  avg d_h= 1382.50116311
mean ddh =  -12.2236608538
step = 196  avg d_h= 1377.91729029
mean ddh =  -12.1831254897
step = 197  avg d_h= 1373.34861824
mean ddh =  -12.1427245671
step = 198  avg d_h= 1368.79509652
mean ddh =  -12.1024576402
step = 199  avg d_h= 1364.25667491
mean ddh =  -12.0623242644
step = 200  avg d_h= 1359.73330331
mean ddh =  -12.0223239969
step = 2

mean ddh =  -7.52655677095
step = 342  avg d_h= 848.462680554
mean ddh =  -7.50159612044
step = 343  avg d_h= 845.649582009
mean ddh =  -7.47671826664
step = 344  avg d_h= 842.845812659
mean ddh =  -7.45192293476
step = 345  avg d_h= 840.051341558
mean ddh =  -7.4272098509
step = 346  avg d_h= 837.266137864
mean ddh =  -7.4025787421
step = 347  avg d_h= 834.490170836
mean ddh =  -7.37802933629
step = 348  avg d_h= 831.723409835
mean ddh =  -7.35356136233
step = 349  avg d_h= 828.965824324
mean ddh =  -7.32917454994
step = 350  avg d_h= 826.217383868
mean ddh =  -7.30486862977
step = 351  avg d_h= 823.478058132
mean ddh =  -7.28064333337
step = 352  avg d_h= 820.747816882
mean ddh =  -7.25649839316
step = 353  avg d_h= 818.026629984
mean ddh =  -7.23243354247
step = 354  avg d_h= 815.314467406
mean ddh =  -7.2084485155
step = 355  avg d_h= 812.611299212
mean ddh =  -7.18454304736
step = 356  avg d_h= 809.91709557
mean ddh =  -7.160716874
step = 357  avg d_h= 807.231826742
mean ddh =  -7

/home/mcoving/anaconda2/lib/python2.7/site-packages/landlab/components/conduit_networks/pressurized_flow.py:159: RuntimeWarning: invalid value encountered in power
  self.Q = np.sign(dh)*(np.fabs(dh)/r)**(1./a)


step = 451  avg d_h= 5.89678652626e+31
mean ddh =  -1.09020192326e+49
step = 452  avg d_h= -4.08825721221e+48
mean ddh =  7.55839787599e+65
step = 453  avg d_h= 2.8343992035e+65
mean ddh =  -5.24025661972e+82
step = 454  avg d_h= -1.9650962324e+82
mean ddh =  3.63308334532e+99
step = 455  avg d_h= 1.36240625449e+99
mean ddh =  -2.51882599496e+116
step = 456  avg d_h= -9.4455974811e+115
mean ddh =  1.7463085181e+133
step = 457  avg d_h= 6.54865694287e+132
mean ddh =  -1.21072017141e+150
step = 458  avg d_h= -4.5402006428e+149
mean ddh =  8.39395397937e+166
step = 459  avg d_h= 3.14773274226e+166
mean ddh =  -5.81954980774e+183
step = 460  avg d_h= -2.1823311779e+183
mean ddh =  4.0347087973e+200
step = 461  avg d_h= 1.51301579899e+200
mean ddh =  -2.7972739502e+217
step = 462  avg d_h= -1.04897773132e+217
mean ddh =  1.93935719913e+234
step = 463  avg d_h= 7.27258949673e+233
mean ddh =  -1.34456131676e+251
step = 464  avg d_h= -5.04210493785e+250
mean ddh =  9.32187807043e+267
step = 46

/home/mcoving/anaconda2/lib/python2.7/site-packages/landlab/components/conduit_networks/pressurized_flow.py:145: RuntimeWarning: overflow encountered in power
  self.r = 8*f*L/(g*r_s**2.*d_h**5.)
/home/mcoving/anaconda2/lib/python2.7/site-packages/landlab/components/conduit_networks/pressurized_flow.py:159: RuntimeWarning: divide by zero encountered in divide
  self.Q = np.sign(dh)*(np.fabs(dh)/r)**(1./a)
/home/mcoving/anaconda2/lib/python2.7/site-packages/landlab/components/conduit_networks/glacier_conduit.py:133: RuntimeWarning: overflow encountered in power
  self.melt = 16.*self.rho_w*f/(np.pi**3.*self.rho_ice*self.L_v)*abs(self._grid.at_link['conduit__discharge'])**3./d_h**6.
/home/mcoving/anaconda2/lib/python2.7/site-packages/landlab/components/conduit_networks/pressurized_flow.py:145: RuntimeWarning: overflow encountered in multiply
  self.r = 8*f*L/(g*r_s**2.*d_h**5.)
/home/mcoving/anaconda2/lib/python2.7/site-packages/landlab/components/conduit_networks/pressurized_flow.py:159

mean ddh =  nan
step = 481  avg d_h= nan
mean ddh =  nan
step = 482  avg d_h= nan
mean ddh =  nan
step = 483  avg d_h= nan
mean ddh =  nan
step = 484  avg d_h= nan
mean ddh =  nan
step = 485  avg d_h= nan
mean ddh =  nan
step = 486  avg d_h= nan
mean ddh =  nan
step = 487  avg d_h= nan
mean ddh =  nan
step = 488  avg d_h= nan
mean ddh =  nan
step = 489  avg d_h= nan
mean ddh =  nan
step = 490  avg d_h= nan
mean ddh =  nan
step = 491  avg d_h= nan
mean ddh =  nan
step = 492  avg d_h= nan
mean ddh =  nan
step = 493  avg d_h= nan
mean ddh =  nan
step = 494  avg d_h= nan
mean ddh =  nan
step = 495  avg d_h= nan
mean ddh =  nan
step = 496  avg d_h= nan
mean ddh =  nan
step = 497  avg d_h= nan
mean ddh =  nan
step = 498  avg d_h= nan
mean ddh =  nan
step = 499  avg d_h= nan
mean ddh =  nan
step = 500  avg d_h= nan
mean ddh =  nan


/home/mcoving/anaconda2/lib/python2.7/site-packages/landlab/components/conduit_networks/glacier_conduit.py:154: RuntimeWarning: overflow encountered in multiply
  ddh = (self.melt - self.creep)*self.dt
